# Week 08

Unsupervised Learning: Distances and Clustering

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/image_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image as PImage

from data_utils import StandardScaler
from data_utils import KMeansClustering, GaussianClustering
from data_utils import object_from_json_url

from image_utils import get_pixels, make_image

## Cost and Distance Functions

The concept of **_distance_** is something that we saw and used in previous exercises but didn't talk too much about.

**_Distance_** is how we tell how close two data points are to each other, and is the basis for clustering, classification and regression algorithms.

In classification, we learn how to label new data based on how "close" it is to our already-labeled training data. In regression, we find parameters to equations that make our line-of-best-fit "close" to all of the points in the dataset. In clustering, we group data points in a way that minimizes distances between points within a cluster, while maximizing the distances between clusters. Distance is also an important concept for recommendation systems where we want to calculate when someone's taste is close to someone else's.

### 1D

The concept of distance in one dimension is pretty easy to understand: it's how far two points on a line are to each other. Physically, we can think of $1D$ distance as the distance between runners in a race, or, we can even think of time as a one-dimensional space where we measure distance between events in seconds, or minutes, or days.

<img src="./imgs/dist1d.jpg" height="220px" />

Each point in 1-dimensional space is described with a single variable, and the distance between any two points is just the absolute value of their difference:

$\displaystyle D(x_0, x_1) = |x_0 - x_1|$<br>
$\displaystyle D(x_0, x_2) = |x_0 - x_2|$

### 2D

Two-dimensional distances, where we have $2$ variables to describe each of our points, is also pretty familiar to us. This can be the distance between two cities on a map, measured in angles of longitude and latitude, or distances between two points in Manhattan, measured in streets and avenues.

<img src="./imgs/dist2d.jpg" width="95%" />

We have $2$ variables for each of our points and we also have $2$ ways in which we can combine them to measure distances in $2D$. The first is called $L1$, or Manhattan, distance, and it's the sum of the distances in each of the separate dimensions.

$\displaystyle D_{L1}(x_0y_0, x_1y_1) = |x_0 - x_1| + |y_0 - y_1|$

The other way of measuring distances in $2D$ is using the $L2$, or Euclidean, distance formula:

$\displaystyle D_{L2}(x_0y_0, x_2y_2) = \sqrt{(x_0 - x_2)^2 + (y_0 - y_2)^2}$

This is pretty easy to understand as distances on a map, but... what if $x$ is a variable for $height$ in our dataset, and $y$ is the variable for $ear\ length$? The calculations are still valid. As long as we remember to normalize our data, we can use the $L1$ or $L2$ formulas to figure out how "close" our data points are.

### 3D

Three-dimensional points have $3$ variables that describe them, and, while less common, it's still easy to understand how to measure the distance between them. We could be talking about the distance between planets, or between atoms, or between a wifi router and a cellphone. As long as the points aren't on a plane, we need $3$ variables to describe them and measure the distance between them.

<img src="./imgs/dist3d.jpg" width="95%" />

We can extend the $L1$ and $L2$ distance formulas to work in $3D$:

$\displaystyle D_{L1}(x_0y_0z_0, x_1y_1z_1) = |x_0 - x_1| + |y_0 - y_1| + |z_0 - z_1|$

$\displaystyle D_{L2}(x_0y_0z_0, x_2y_2z_2) = \sqrt{(x_0 - x_2)^2 + (y_0 - y_2)^2 + (z_0 - z_2)^2}$

This works even when our $3$ variables aren't actually physical locations. If $x$ is a variable that keeps track of the number of rooms in a house, $y$ a variable for the age of the house, and $z$ the total area of the house, we can use the above formulas to measure how "close" two houses in our dataset are (after we normalize our data, of course).

### N-Dimensions

Most of the datasets we've seen so far already have more then $3$ features/dimensions... what then?

Well... the $L1$ and $L2$ distance formulas can be used regardless of the number of features/dimensions in our dataset. We just keep adding parameters to our formula:

$\displaystyle D_{L1} = \sum_{d}{|A_d - B_d|}$ (for all dimensions $d$)

$\displaystyle D_{L2} = \sqrt{\sum_{d}{(A_d - B_d)^2}}$ (for all dimensions $d$)

So even when we have a dataset with $15$ or $20$ features/dimensions, we can still get some idea of how "close" two points in that dataset are.

### Other Distance Formulas

$L1$ and $L2$ are definitely the most widely used distance formulas in ML applications, but they aren't the only ones. Two other types of distances are:

#### Cosine Similarity

When we're dealing with datasets that are very sparse (there are more dimensions than points), and the $L1$ and $L2$ distances that separate the data points are really huge, we might want to measure the cosine similarity between two points instead.

<img src="./imgs/distcos.jpg" height="300px" />

In the drawing above, instead of measuring the direct distances between $x_0y_0$, $x_1y_1$ and $x_2y_2$, we can pick a separate reference point and measure the cosine of the angles formed by lines drawn from the reference point to each of the other points. We can see that angle $\theta_{12}$ is smaller than $\theta_{01}$ and $\theta_{02}$, which means that $(x_1y_1, x_2y_2)$ is the pair of most similar points.

Points with cosine values close to $1$ are in the same direction in space; points with cosine values close to $0$ are in perpendicular directions, and points with cosine values close to $-1$ are in opposite directions.

$\displaystyle cos(A, B) = \frac{A \cdot B}{ \left|\left|A\right|\right| \left|\left|B\right|\right|}$

$\displaystyle cos(x_0y_0, x_1y_1) = \frac{x_0x_1 + y_0y_1}{\sqrt{x_0^2+y_0^2} \sqrt{x_1^2+y_1^2}}$

#### Mahalanobis Distance

This is useful for measuring distances between points that are part of a collection of points.

<img src="./imgs/distmana-01.jpg" height="250px" />

In the drawing above, if we only had the points $x_0y_0$, $x_1y_1$ and $x_2y_2$, we could use $L2$ distances and everything is fine:

$\displaystyle D_{L2}(x_0y_0, x_1y_1) > D_{L2}(x_0y_0, x_2y_2)$.

But, if instead, $x_0y_0$, $x_1y_1$ and $x_2y_2$ are part of a collection of points with a well-defined average and standard deviation, like the image below, we should use something that makes more common distances shorter, and rarer distances larger.

<img src="./imgs/distmana-02.jpg" height="250px" />

In this case, where $x_0y_0$ and $x_2y_2$ are on the extremes of the distribution, we want the distance between them to be larger than the distance between $x_0y_0$ and $x_1y_1$, which happens along a more common direction of the data.

In order to have $\displaystyle D_{M}(x_0y_0, x_2y_2) > D_{M}(x_0y_0, x_1y_1)$, we have to take into account the distribution of our data: its mean, standard deviation and covariances. One way to do that is using the formula below:

$\displaystyle D_{M}(A, B) = \sqrt{(A - B)^2 V_I}$.

Where $V_I$ is the inverse of the covariance matrix of our points.

## Unsupervised Learning

### Clustering

#### More Wine ! 🍷🍷🍷

Let's pretend we own an online wine store.

Last week we created a model that predicts wine quality based on a bunch of its properties. We could use this model to figure out how much to pay suppliers for the wine, and how much to charge costumers.

But, maybe this "`quality`" feature might not be something we want to share with our costumers. Even though it's based on data, it sounds abstract and subjective and would require explanations about our data and our process, which could create confusion.

Using all features from the original dataset (`alcohol`, `acidity`, `density`, etc) might also not be very useful for costumers who want to buy new wines that are similar to ones that they have previously liked.

What we can do instead is classify the wines into groups that take into account all of the features of the dataset, but present costumers with a more manageable amount of information.

### Recommendations

What we're really hoping to have is a simple recommendation system for our costumers, where we can recommend wines based on previous wines they liked, without them having to know the $6$ features of the previous wines.

There are a few ways of doing this, but the strategy we'll take is called clustering.

### Clustering

[Clustering](https://en.wikipedia.org/wiki/Cluster_analysis), or cluster analysis, is an example of an *unsupervised* learning method that groups items based on their many features and properties.

We'll use it to divide our wines in such a way that wines in the same group, or *cluster*, are more similar to each other than to wines in other clusters.

These clusters won't necessarily correlate directly to the features in our dataset, but will be computed using a combination of the features.

### Supervised Learning

The models that we've trained so far for doing regression and classification are considered *supervised* models. During training we give the model our input features, but also provide it with the *correct* values for the output signals. These output signals tend to be human-labeled values, and are sometimes called the *supervisory signals*.

When fully-labeled training data is processed during training, we are hoping that the model learns to extrapolate what it *sees* in the labeled data to new, unseen, unlabeled instances of data with the same input features, but unknown output values.

#### Supervised Classification:

Given a set of initial data points with labels:<br>
<img src="./imgs/classification-02.jpg" width="620px"/>

We create a model that learns to assign labels to the original points:<br>
<img src="./imgs/classification-03.jpg" width="620px"/>

so that later we can assign correct labels to new data points:<br>
<img src="./imgs/classification-04.jpg" width="620px"/>

### Unsupervised Learning

Unlike supervised learning, unsupervised models learn patterns from unlabeled data. This means all of the features are considered input features, and there are no separate output features or signals. The idea is that by analyzing and processing data in specific ways, the model is able to build a concise representation of its features and create new ways of interpreting, visualizing or generating similar data.

We can use unsupervised learning models to explore new datasets and try to simplify our data before we do any kind of supervised learning.

We can also use supervised learning to build recommendation systems that learn how to group items by their many features or characteristics.

The steps for training an unsupervised model should seem familiar:

1. Load dataset
2. Encode label features as numbers
3. Normalize the data
4. Select variables and features to be considered
5. Create a model
6. Run model on input data and test data
7. Measure error

Even though it all looks familiar, that last step isn't very obvious.

How do we measure error on a model that doesn't have a set of correct answers?

Maybe *error* is not the right term, but we'll see how to define *metrics* to score and measure our unsupervised models.

#### Unsupervised Clusterings:
Since there are no correct labels, both of the following clusterings are valid!

<img src="./imgs/clustering-00.jpg" width="620px"/>

<img src="./imgs/clustering-01.jpg" width="620px"/>

Let's run it !

### Preparing Data

We'll load the same wine dataset as last week and normalize its features:

In [ ]:
## 1. Load Dataset
WINE_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/wines.json"

# Read into DataFrame
wines_data = object_from_json_url(WINE_FILE)
wines_df = pd.DataFrame.from_records(wines_data)

## 3. Normalize
wine_scaler = StandardScaler()
wines_scaled = wine_scaler.fit_transform(wines_df)

## 4. Select variables to be considered
##    We're gonna drop the quality features to avoid re-clustering by quality
features = wines_scaled.drop(columns=["quality"])

### Clusterings

Let's look at our first clustering algorithm:

#### [K-means Clustering](https://scikit-learn.org/stable/modules/clustering.html#k-means):
Tries to separate the data into $k$ groups with similar properties. Requires the number of clusters to be determined beforehand, and the algorithm tries to minimize the difference between objects in a cluster.

In [ ]:
n_clusters = 4

## 5. Create Clustering object
km_model = KMeansClustering(n_clusters=n_clusters)

## 6. Run the model on the data
km_predicted = km_model.fit_predict(features)

### Plots

Let's pick $2$ or $3$ variables to visualize our data and clusters.

This could be any of our features, but let's look at the *covariances* table and pick features related to the highest covariance magnitudes.

In [ ]:
## Look at covariances of features
features.cov()

In [ ]:
## TODO: look at max absolute value per column

The highest pairs are `density`/`sugar` and `alcohol`/`density`.

We should visualize our clusters as a function of `density`, `sugar` and `alcohol`.

Let's define a reusable function to help us with the plotting:

In [ ]:
def plot_clusters(labels, clusters, title):
  xl, yl, zl = labels[:3]
  x = wines_scaled[xl]
  y = wines_scaled[yl]
  z = wines_scaled[zl]

  # 2D
  plt.scatter(x, y, c=clusters, marker='o', linestyle='', alpha=0.5)
  plt.title(f"{title} clustering")
  plt.xlabel(xl)
  plt.ylabel(yl)
  plt.ylim([-2.2, 3])
  plt.show()

  plt.scatter(x, z, c=clusters, marker='o', linestyle='', alpha=0.5)
  plt.title(f"{title} clustering")
  plt.xlabel(xl)
  plt.ylabel(zl)
  plt.ylim([-2.2, 3])
  plt.show()

  # 3D
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(projection='3d')

  ax.scatter(x, y, z, c=clusters, marker='o', linestyle='', alpha=0.5)

  ax.set_title(f"{title} clustering")
  ax.set_xlabel(xl)
  ax.set_ylabel(yl)
  ax.set_zlabel(zl)
  ax.set_ylim(-2.5, 8)
  ax.set_zlim(-2.5, 2.5)

  plt.show()

In [ ]:
# For plotting
labels = ["alcohol", "density", "sugar"]
clusters = km_predicted["clusters"]

plot_clusters(labels, clusters, "K-Means")

## Number of clusters

Does the number of clusters affect clustering ?

Change the variable and re-run clustering to see how the groupings change.

In [ ]:
# TODO: change n_clusters above and re-run clustering

### Interpretation

What changes ? Does one choice seem better than the others ?

### Other Clustering Options

There are many algorithms for clustering data, that differ in the assumptions they make about the data, and which parameters should be optimized.

There's one method called [Gaussian Clustering](https://scikit-learn.org/stable/modules/mixture.html#mixture) that is similar to K-means, but it assumes that all features of our data can be modeled as [Gaussian distributions](https://en.wikipedia.org/wiki/Normal_distribution). During clustering, the algorithm incorporates information about the covariance structure of the data to try to determine mean and standard deviation values for each cluster.

[Scikit-Learn](https://scikit-learn.org/) has implementation for this algorithm, and many more. It's called `GaussianClustering` and its constructor take the same parameters as the `KMeansClustering` constructor.

Repeat steps $5$ and $6$ to create a model using `GaussianClustering` and run it on our data.

In [ ]:
n_clusters = 4

## TODO: 5. Create Clustering object
gc_model = ''

## TODO: 6. Run the model on the data
gc_predicted = ''

### Plot results

In [ ]:
# Plots for Gaussian Clustering results
labels = ["alcohol", "density", "sugar"]
clusters = gc_predicted["clusters"]

plot_clusters(labels, clusters, "Gaussian")

### Interpretation

Are the results any different than k-mean clustering ?

### Scoring

Would be nice to have a way to measure how good these clusters actually are.

It would help determine if we need more clusters, or if one method is actually better than the other.

There are a couple of ways to do this. We'll look at three of them.

### Distance

The first kind of scoring uses the sum of the distances between each point and its cluster's center as a metric.

Each cluster's center is represented by the average values of all of the features of all of its members: $(\overline{F_0}, \overline{F_1}, \overline{F_2}, ...)$. Once we know that we can use the L2-distance we saw above to calculate and accumulate the distances from each point to its cluster's center.

A smaller cluster distance means that the cluster center is a good representation of its members.

Luckily, our clustering models have a `distance_error()` function that can be used to report the distance error, after `fit()` has been called.

In [ ]:
print("KMeans distance error:", km_model.distance_error())
print("Gaussian distance error:", gc_model.distance_error())

### Likelihood

The second way of scoring clusters treats each cluster as a potential normal distribution, and then calculates the likelihood that each point came from its cluster distribution.

Values closer to zero mean that the clusters' statistical properties (mean, variation) are good estimators for the data.

Our model objects also have a `likelihood_error()` function we can use:

In [ ]:
print("KMeans likelihood error:", km_model.likelihood_error())
print("Gaussian likelihood error:", gc_model.likelihood_error())

Although somewhat related, the `distance` and `likelihood` calculations measure different things, and are in different units.

We can't compare distances to likelihoods to draw any kind of conclusion.

What we want to do is use either one of these metrics to select a clustering method and tune its parameters.

### Balance

A final metric we can consider when analyzing different clustering algorithms and strategies is to see how balanced the resulting clusters are. This isn't always important; we might have categories of items or events that are more common than others, and will produce unequal cluster groups.

In other cases, where we know we want to have groups of similar sizes, this is a good metric to look at. For example, if we were to use the body measurement dataset for deciding how many sizes of bike helmets to produce, we should probably have sizes that cover similar portions of the population, and avoid very bespoke sizes that only fit few people.

We compute `balance error` by summing the differences between our cluster sizes and the sizes of a perfectly balanced clustering. Once we have this sum, we scale it to get a number between $0$, for a perfectly balanced clustering, and $1$, for a most-unbalanced clustering.

$\displaystyle balance\ error = \frac{1}{2} \left(\frac{n}{n-1}\right) \sum_{i=1}^{n}{\left|\frac{C_i}{C_0 + C_1 + ... + C_n} - \frac{1}{n}\right|}$

The $\frac{C_i}{C_0 + C_1 + ... + C_n}$ terms are the sizes of our $n$ clusters expressed as the percentage of the total number of items in all clusters. The $\frac{1}{n}$ term is the size of each cluster in a perfectly balanced clustering. We sum up these differences and scale it all by $\frac{1}{2} \left(\frac{n}{n-1}\right)$ to get a number between $0$ and $1$.

We don't have to focus too much on this math right now. It's here for completeness and because it's good to practice reading an algorithm described as text, math equations and code.

### Balance Error

Luckily this has also been implemented for us and we can get our model's `balance error` by calling the `balance_error()` function of our clustering object:

In [ ]:
print("KMeans balance error:", km_model.balance_error())
print("Gaussian balance error:", gc_model.balance_error())

### Number of clusters

If we consider the distance and balance metrics for the $2$ algorithms, it seems like `KMeansClustering` produces more dense and balanced clusters. This suggests that our features might not be normally distributed, or, our data is not a combination of gaussian features.

Let's try different cluster numbers for the K-Means algorithm to see if there's a *better* way of clustering our wines:

In [ ]:
# try 2 - 10 clusters for K-Means Clustering
num_clusters = list(range(2,10))

# collect distance, likelihood and balance errors
dist_err = []
like_err = []
bala_err = []

# get distance, likelihood and balance for different clustering sizes
for n in num_clusters:
  mm = KMeansClustering(n_clusters=n)
  mm.fit_predict(features)
  dist_err.append(mm.distance_error())
  bala_err.append(mm.balance_error())

# plot errors as function of number of clusters
plt.plot(num_clusters, dist_err, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Distance Error")
plt.title("K-means Clustering")
plt.ylim([0, 3])
plt.show()

plt.plot(num_clusters, bala_err, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Balance Error")
plt.title("K-means Clustering")
plt.ylim([0, 1])
plt.show()

### Interpretation

Doesn't seem like the number of clusters affects the distance or balance errors for `KMeansClustering` too much.

Let's re-cluster into $2$ clusters, just to see what that looks like:

In [ ]:
# Cluster
n = 2
m_model = KMeansClustering(n_clusters=n)
km_predicted = m_model.fit_predict(features)

# Plot
labels = ["alcohol", "density", "sugar"]
clusters = km_predicted["clusters"]

plot_clusters(labels, clusters, "K-Mean")

### Analysis

So, even though a higher number of clusters gives us slightly smaller distance error values, some of the clusters are really small and hard to find on the graphs.

If this clustering is to be used for recommending wines to costumers, maybe using $2$ or $3$ clusters is a more sensible way of grouping our wines because the subtleties between $5$ categories of wine might be less easy to explain.

Using $2$ or $3$ categories is probably more legible. The categories could be something like: `bold` and `sweet`.

## Images

Clustering can also be used to help analyze image and audio files.

We can extract color information from an image by clustering its pixels by their RGB values. This can be used to get an estimate of the most "important" colors in an image. They're not the most common colors, necessarily, but the colors necessary to represent the image.

This is called `color quantization` and is a kind of compression because we reduce the total number of colors in an image from a possible $16\text{,}581\text{,}375$ unique colors to $4$, $8$, $16$, etc... colors while preserving the overall appearance of the image. The calculated cluster centers become the color palette of the image, and we can re-color the image using only those colors.

We start by loading an image:

In [ ]:
mimg = PImage.open("./data/image/arara.jpg")

display(mimg)

### Colors as features

Since this process works on the pixels of a single image, we can think of each pixel as a measurement and its `R`, `G` and `B` values as its features.

Before we put our image through clustering we should turn it into a `DataFrame` where the rows are the pixels and the columns are the `R`, `G`, `B` values.

We can use the `pd.DataFrame.from_records()` function, but first we have to turn our pixel list into a list of objects:

`[ [R,G,B], [R,G,B], [R,G,B], ... ]` -> `[{"R": R, "G": G, "B": B}, {"R": R, "G": G, "B": B}, ...]`

In [ ]:
# Create list of objects from list of lists
pixelObj = [{"R":r, "G":g, "B":b} for r,g,b in get_pixels(mimg)]

img_df = pd.DataFrame.from_records(pixelObj)

img_df.head()

### Encoding/Scaling

Not needed! All our features are already numbers, and they're all in the same units.

### Plot

We just opened a new "dataset"... let's plot it.

Since our features are the `R`, `G` and `B` channel values for each pixel, we can actually plot these in $3D$.

Each of the features will place a point in `x`, `y`, `z` space, and the color of the point can be the color of the pixel.

The only funny business here is that `pyplot` expects colors in the range $[0, 1]$ and not $[0, 255]$, so we have to convert those.

We could use a `MinMaxScaler`, but since the inputs are already in a known range and the math is easy, we'll just use a comprehension.

In [ ]:
# convert [0, 255] to [0, 1]
c = [(r/255, g/255, b/255) for r,g,b in get_pixels(mimg)]

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

ax.scatter(img_df["R"], img_df["G"], img_df["B"], c=c, marker='o', linestyle='', alpha=0.5)
ax.set_xlabel("R")
ax.set_ylabel("G")
ax.set_zlabel("B")
plt.show()

### Repeat

Repeat the above process for a different image in the `./data/image` directory, or something else.

In [ ]:
# TODO: Visualize pixel color distributions for other images

### Cluster !

Set up the `Clustering` object and run `fit_predtict()`.

In [ ]:
n_clusters=8

## Create Clustering object
km_model = KMeansClustering(n_clusters=n_clusters)

## Run the model on the pixel data
km_predicted = km_model.fit_predict(img_df)

km_predicted.head()

### Un-Cluster / Group

The `km_predicted` variable holds a `DataFrame` that is a mapping from pixel index to cluster index, but what we really want is to re-map our original pixels into the color palette made up of our cluster centers.

The `KMeansClustering` object has a member variable called `cluster_centers_` that holds the centers of our clusters. We can use this to build a new pixel array for our image.

Let's take a look at the cluster centers:

In [ ]:
km_model.cluster_centers_

We have $8$ lists inside the `cluster_centers_` list. They should represent colors, but they're using floating point numbers right now, which will most likely give us troubles when we try to turn these into an image.

Let's transform these into `int`s:

In [ ]:
# using round() or int() to ensure our cluster centers are valid color values (ints)
color_centers = [[round(r), round(g), round(b)] for r,g,b in km_model.cluster_centers_]

print(color_centers)

Now we can use these in a pixel array.

Let's iterate through the `km_predicted["clusters"]` and use those values to push their corresponding cluster center colors into a new pixel array.

We go through `km_predicted["clusters"]`, and if we see cluster $0$ we want to push `color_centers[0]` onto our pixel array, etc...

In [ ]:
clustered_pxs = []

# Iterate through the cluster list and append the right color for each pixel
for gidx in km_predicted["clusters"]:
  clustered_pxs.append(color_centers[gidx])

display(make_image(clustered_pxs, mimg.size[0]))